In [28]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [29]:
# Model
m = gp.Model("district")
m

<gurobi.Model Continuous instance district: 0 constrs, 0 vars, No parameter changes>

In [30]:
state = 'WA'
population = "districting_2020\\{}\\counties\\graph\\{}.population".format(state, state)
dimacs = "districting_2020\\{}\\counties\\graph\\{}.dimacs".format(state, state)

with open('Numberofdistricts.txt', 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts = int(parts[1])


with open(population, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)

num_districts


10

In [31]:
## Reading distances from file
filename = "districting_2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)
d = np.genfromtxt(filename, delimiter=',', skip_header=1)
d = d[:, 1:]

## Reading populations from file
with open(population, 'r') as file:
    # Read the total population from the first line
    total_population_line = next(file).strip()  # Removes newline character at the end
    total_population = int(total_population_line.split('=')[1].strip())
    
    # Initialize a dictionary to store the rest of the data
    p = {}
    
    # Iterate over the remaining lines
    for line in file:
        # Each line is expected to have two values, separated by a space
        key, value = line.strip().split()  # Split by whitespace
        p[int(key)] = int(value)  # Convert both to integers and store in the dictionary

## Reading neighbors from file
neighbors = set()
with open(dimacs, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('e'):
            _, node1, node2 = line.split()
            node1 = int(node1)
            node2 = int(node2)
            if (node1, node2) not in neighbors and (node2, node1) not in neighbors:
                neighbors.add((node1, node2))

list(neighbors)

[(16, 20),
 (18, 35),
 (13, 17),
 (13, 26),
 (2, 32),
 (9, 28),
 (15, 16),
 (8, 32),
 (19, 32),
 (13, 28),
 (15, 25),
 (24, 28),
 (11, 37),
 (2, 34),
 (7, 30),
 (29, 36),
 (31, 33),
 (9, 12),
 (12, 29),
 (4, 34),
 (15, 18),
 (20, 35),
 (4, 18),
 (22, 32),
 (19, 36),
 (10, 33),
 (12, 24),
 (23, 24),
 (23, 33),
 (14, 30),
 (4, 38),
 (19, 29),
 (10, 26),
 (2, 22),
 (2, 31),
 (15, 34),
 (20, 30),
 (21, 29),
 (8, 31),
 (11, 36),
 (6, 15),
 (3, 7),
 (6, 33),
 (12, 28),
 (14, 25),
 (23, 28),
 (27, 28),
 (2, 8),
 (5, 34),
 (8, 33),
 (11, 29),
 (0, 38),
 (32, 36),
 (20, 25),
 (21, 24),
 (12, 21),
 (29, 37),
 (18, 34),
 (22, 34),
 (1, 11),
 (11, 22),
 (6, 10),
 (15, 31),
 (8, 19),
 (22, 36),
 (9, 27),
 (13, 27),
 (7, 20),
 (1, 34),
 (15, 33),
 (18, 20),
 (26, 33),
 (18, 38),
 (2, 5),
 (19, 21),
 (5, 31),
 (28, 33),
 (0, 35),
 (6, 14),
 (16, 25),
 (35, 38),
 (5, 15),
 (17, 26),
 (8, 23),
 (19, 23),
 (16, 18),
 (26, 28),
 (6, 25),
 (25, 30),
 (21, 23),
 (3, 20),
 (1, 22)]

In [32]:
adjacency_matrix_file = r"districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)

# Set decision variable x for each node i whether it is in district j
x = {}
for i in range(num_counties):
    for j in range(num_districts):
        x[i, j] = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

## Set decision variable y for each edge in neighbors, between nodes i and k, for each district j
y = {}
for i, k in neighbors:
    for j in range(num_districts):
        y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# for i in range(num_counties):
#     for k in range(num_counties):
#         for j in range(num_districts):
#             y[i, k, j] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# Set slack variable
slack = {}
for j in range(num_districts):
    slack[j] = m.addVar(obj=1, name="slack")




alpha = 1  # Weight of the balanced weight term
y


{(16, 20, 0): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 1): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 2): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 3): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 4): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 5): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 6): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 7): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 8): <gurobi.Var *Awaiting Model Update*>,
 (16, 20, 9): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 0): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 1): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 2): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 3): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 4): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 5): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 6): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 7): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 8): <gurobi.Var *Awaiting Model Update*>,
 (18, 35, 9)

In [33]:
# Set objective function
m.setObjective(gp.quicksum(y[i, k, j] * d[i, k] for i, k in neighbors for j in range(num_districts)) \
             + alpha * (gp.quicksum(slack[j] for j in range(num_districts))), gp.GRB.MINIMIZE)


In [34]:
## Add constraints

# Each edge in neighbors is assigned to exactly one district
for i, k in neighbors:
    for j in range(num_districts):
        m.addConstr(x[i, j] + x[k, j] >= 2 * y[i, k, j] )

# Add population constraint with slack variable
for j in range(num_districts):
    m.addConstr(gp.quicksum(x[i, j] * p[i] for i in range(num_counties)) >= ((total_population / num_districts) - slack[j]))
    m.addConstr(gp.quicksum(x[i, j] * p[i] for i in range(num_counties)) <= ((total_population / num_districts) + slack[j]))

# Each county is assigned to exactly one district
for i in range(num_counties):
    m.addConstr(gp.quicksum(x[i, j] for j in range(num_districts)) == 1)



In [35]:
# Write LP formulation to a file
m.write("districts.lp")

# Read the LP formulation from the file
with open("districts.lp", "r") as f:
    lp_formulation = f.read()

# Print LP formulation
print("LP Formulation:")
print(lp_formulation)

# Solve
m.optimize()

LP Formulation:
\ Model district
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  80 y_16_20_0 + 80 y_16_20_1 + 80 y_16_20_2 + 80 y_16_20_3 + 80 y_16_20_4
   + 80 y_16_20_5 + 80 y_16_20_6 + 80 y_16_20_7 + 80 y_16_20_8
   + 80 y_16_20_9 + 76 y_18_35_0 + 76 y_18_35_1 + 76 y_18_35_2
   + 76 y_18_35_3 + 76 y_18_35_4 + 76 y_18_35_5 + 76 y_18_35_6
   + 76 y_18_35_7 + 76 y_18_35_8 + 76 y_18_35_9 + 49 y_13_17_0
   + 49 y_13_17_1 + 49 y_13_17_2 + 49 y_13_17_3 + 49 y_13_17_4
   + 49 y_13_17_5 + 49 y_13_17_6 + 49 y_13_17_7 + 49 y_13_17_8
   + 49 y_13_17_9 + 62 y_13_26_0 + 62 y_13_26_1 + 62 y_13_26_2
   + 62 y_13_26_3 + 62 y_13_26_4 + 62 y_13_26_5 + 62 y_13_26_6
   + 62 y_13_26_7 + 62 y_13_26_8 + 62 y_13_26_9 + 85 y_2_32_0 + 85 y_2_32_1
   + 85 y_2_32_2 + 85 y_2_32_3 + 85 y_2_32_4 + 85 y_2_32_5 + 85 y_2_32_6
   + 85 y_2_32_7 + 85 y_2_32_8 + 85 y_2_32_9 + 106 y_9_28_0 + 106 y_9_28_1
   + 106 y_9_28_2 + 106 y_9_28_3 + 106 y_9_28_4 + 106 y_9_28_5
   + 106 y_9_

In [36]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 979 rows, 1320 columns and 3950 nonzeros
Model fingerprint: 0x52facae7
Variable types: 10 continuous, 1310 integer (1310 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]
Presolved: 59 rows, 400 columns, 1190 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 8
  MIR: 1
  RLT: 8

Explored 24 nodes (598 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 7: 3.41436e+06 3.59592e+06 3.66208e+06 ... 5.34356e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 3.414355400000e+06, best bound 3.414355400000e+06, gap 0.0000%


In [37]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nObjective Value: %g' % m.ObjVal)
        for i in range(num_counties):
            for j in range(num_districts):
                print("x_{}_{} = {}".format(i, j, x[i, j].X))
        for j in range(num_districts):
            print("slack_{} = {}".format(j, slack[j].X))
    else:
        print('No solution')

printSolution()


Objective Value: 3.41436e+06
x_0_0 = 0.0
x_0_1 = 0.0
x_0_2 = 0.0
x_0_3 = 1.0
x_0_4 = 0.0
x_0_5 = 0.0
x_0_6 = 0.0
x_0_7 = 0.0
x_0_8 = 0.0
x_0_9 = 0.0
x_1_0 = 0.0
x_1_1 = 0.0
x_1_2 = 1.0
x_1_3 = 0.0
x_1_4 = 0.0
x_1_5 = 0.0
x_1_6 = 0.0
x_1_7 = 0.0
x_1_8 = 0.0
x_1_9 = 0.0
x_2_0 = 0.0
x_2_1 = 0.0
x_2_2 = 0.0
x_2_3 = 0.0
x_2_4 = 0.0
x_2_5 = 0.0
x_2_6 = 0.0
x_2_7 = 1.0
x_2_8 = 0.0
x_2_9 = 0.0
x_3_0 = 0.0
x_3_1 = 0.0
x_3_2 = 1.0
x_3_3 = 0.0
x_3_4 = 0.0
x_3_5 = 0.0
x_3_6 = 0.0
x_3_7 = 0.0
x_3_8 = 0.0
x_3_9 = 0.0
x_4_0 = 0.0
x_4_1 = 0.0
x_4_2 = 0.0
x_4_3 = 0.0
x_4_4 = 1.0
x_4_5 = 0.0
x_4_6 = 0.0
x_4_7 = 0.0
x_4_8 = 0.0
x_4_9 = 0.0
x_5_0 = 0.0
x_5_1 = 0.0
x_5_2 = 0.0
x_5_3 = 0.0
x_5_4 = 0.0
x_5_5 = 0.0
x_5_6 = 0.0
x_5_7 = 1.0
x_5_8 = 0.0
x_5_9 = 0.0
x_6_0 = 0.0
x_6_1 = 0.0
x_6_2 = 1.0
x_6_3 = 0.0
x_6_4 = 0.0
x_6_5 = 0.0
x_6_6 = 0.0
x_6_7 = 0.0
x_6_8 = 0.0
x_6_9 = 0.0
x_7_0 = 0.0
x_7_1 = 0.0
x_7_2 = 0.0
x_7_3 = 0.0
x_7_4 = 0.0
x_7_5 = 0.0
x_7_6 = 0.0
x_7_7 = 1.0
x_7_8 = 0.0
x_7_9 = 0.0
x_8_0 = 0.

In [38]:
y

{(16, 20, 0): <gurobi.Var y_16_20_0 (value 0.0)>,
 (16, 20, 1): <gurobi.Var y_16_20_1 (value 0.0)>,
 (16, 20, 2): <gurobi.Var y_16_20_2 (value 0.0)>,
 (16, 20, 3): <gurobi.Var y_16_20_3 (value 0.0)>,
 (16, 20, 4): <gurobi.Var y_16_20_4 (value 0.0)>,
 (16, 20, 5): <gurobi.Var y_16_20_5 (value 0.0)>,
 (16, 20, 6): <gurobi.Var y_16_20_6 (value 0.0)>,
 (16, 20, 7): <gurobi.Var y_16_20_7 (value 0.0)>,
 (16, 20, 8): <gurobi.Var y_16_20_8 (value 0.0)>,
 (16, 20, 9): <gurobi.Var y_16_20_9 (value 0.0)>,
 (18, 35, 0): <gurobi.Var y_18_35_0 (value 0.0)>,
 (18, 35, 1): <gurobi.Var y_18_35_1 (value 0.0)>,
 (18, 35, 2): <gurobi.Var y_18_35_2 (value 0.0)>,
 (18, 35, 3): <gurobi.Var y_18_35_3 (value 0.0)>,
 (18, 35, 4): <gurobi.Var y_18_35_4 (value 0.0)>,
 (18, 35, 5): <gurobi.Var y_18_35_5 (value 0.0)>,
 (18, 35, 6): <gurobi.Var y_18_35_6 (value 0.0)>,
 (18, 35, 7): <gurobi.Var y_18_35_7 (value 0.0)>,
 (18, 35, 8): <gurobi.Var y_18_35_8 (value 0.0)>,
 (18, 35, 9): <gurobi.Var y_18_35_9 (value 0.0)>,
